In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Do not try to mask unused channels to optimize the code: we have tried it and it was in fact COUNTER-PRODUCTIVE.
# Python is the bottleneck with 8 channels, not numpy, and it does not matter whether we use all 8 or 0 channels.

def shift_numpy(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result


class FIR:
    def __init__(self, nb_channels, coefficients, buffer=None):
        
        self.coefficients = np.expand_dims(np.array(coefficients), axis=1)
        self.taps = len(self.coefficients)
        self.nb_channels = nb_channels
        self.buffer = np.array(z) if buffer is not None else np.zeros((self.taps, self.nb_channels))
    
    def filter(self, x):
        self.buffer = shift_numpy(self.buffer, 1, x)
        filtered = np.sum(self.buffer * self.coefficients, axis=0)
        return filtered

    
class FilterPipeline:
    def __init__(self, nb_channels, power_line_fq=60):
        self.nb_channels = nb_channels
        assert power_line_fq in [50, 60], f"The only supported power line frequencies are 50Hz and 60Hz"
        if power_line_fq == 60:
            self.notch_coeff1 = -0.12478308884588535
            self.notch_coeff2 = 0.98729186796473023
            self.notch_coeff3 = 0.99364593398236511
            self.notch_coeff4 = -0.12478308884588535
            self.notch_coeff5 = 0.99364593398236511
        else:
            self.notch_coeff1 = -0.61410695998423581
            self.notch_coeff2 =  0.98729186796473023
            self.notch_coeff3 = 0.99364593398236511
            self.notch_coeff4 = -0.61410695998423581
            self.notch_coeff5 = 0.99364593398236511
        self.dfs = [np.zeros(self.nb_channels), np.zeros(self.nb_channels)]
        
        self.moving_average = None
        self.moving_variance = np.zeros(self.nb_channels)
        self.ALPHA_AVG = 0.1
        self.ALPHA_STD = 0.001
        self.EPSILON = 0.000001
            
        self.fir_30_coef = [
            0.001623780150148094927192721215192250384,
            0.014988684599373741992978104065059596905,
            0.021287595318265635502275046064823982306,
            0.007349500393709578957568417933998716762,
            -0.025127515717112181709014251396183681209,
            -0.052210507359822452833064687638398027048,
            -0.039273839505489904766477593511808663607,
            0.033021568427940004020193498490698402748,
            0.147606943281569008563636202779889572412,
            0.254000252034505602516389899392379447818,
            0.297330876398883392486283128164359368384,
            0.254000252034505602516389899392379447818,
            0.147606943281569008563636202779889572412,
            0.033021568427940004020193498490698402748,
            -0.039273839505489904766477593511808663607,
            -0.052210507359822452833064687638398027048,
            -0.025127515717112181709014251396183681209,
            0.007349500393709578957568417933998716762,
            0.021287595318265635502275046064823982306,
            0.014988684599373741992978104065059596905,
            0.001623780150148094927192721215192250384]
        self.fir = FIR(self.nb_channels, self.fir_30_coef)
        
    def filter(self, value):
        """
        value: a numpy array of shape (data series, channels)
        """
        for i, x in enumerate(value):  # loop over the data series
            # FIR:
            x = self.fir.filter(x)
            # notch:
            denAccum = (x - self.notch_coeff1 * self.dfs[0]) - self.notch_coeff2 * self.dfs[1]
            x = (self.notch_coeff3 * denAccum + self.notch_coeff4 * self.dfs[0]) + self.notch_coeff5 * self.dfs[1]
            self.dfs[1] = self.dfs[0]
            self.dfs[0] = denAccum
            # standardization:
            if self.moving_average is not None:
                delta = x - self.moving_average
                self.moving_average = self.moving_average + self.ALPHA_AVG * delta
                self.moving_variance = (1 - self.ALPHA_STD) * (self.moving_variance + self.ALPHA_STD * delta**2)
                moving_std = np.sqrt(self.moving_variance)
                x = (x - self.moving_average) / (moving_std + self.EPSILON)
            else:
                self.moving_average = x
            value[i] = x
        return value

In [ ]:
duration = 1
fsample = 250
f1 = 15
f2 = 50
f3 = 60
f4 = 100
f5 = 70
f6 = 80
f7 = 90
scale = 4.0e-5

w1 = 2*np.pi*f1
w2 = 2*np.pi*f2
w3 = 2*np.pi*f3
w4 = 2*np.pi*f4
w5 = 2*np.pi*f5
w6 = 2*np.pi*f6
w7 = 2*np.pi*f7
nb_samples = int(duration*fsample)

sig1 = np.array([np.sin(w1*i/fsample) for i in range(nb_samples)])
sig2 = np.array([np.sin(w2*i/fsample) for i in range(nb_samples)])
sig3 = np.array([np.sin(w3*i/fsample) for i in range(nb_samples)])
sig4 = np.array([np.sin(w4*i/fsample) for i in range(nb_samples)])
sig5 = np.array([np.sin(w5*i/fsample) for i in range(nb_samples)])
sig6 = np.array([np.sin(w6*i/fsample) for i in range(nb_samples)])
sig7 = np.array([np.sin(w7*i/fsample) for i in range(nb_samples)])
sig8 = sig1 + sig2 + sig3 + sig4 + sig5 + sig6 + sig7

v = np.array([sig1, sig2, sig3, sig4, sig5, sig6, sig7, sig8]).T * scale

mask = [0,0,0,0,0,0,0,1]

v.shape

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,5))
plt.plot(v[:, 7])

In [ ]:
import time
print(mask)
fp = FilterPipeline(nb_channels=8, power_line_fq=60)

ts = time.time()
v = fp.filter(v)
print(time.time() - ts)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(v[:, 7])